# Creando el primer gráfico interactivo

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

df = pd.read_csv("data/Inmigrantes_Canadá.csv")
df.set_index("Pais", inplace=True)

anos = list(map(str, range(1980, 2014)))
chile = df.loc["Chile", anos]
cl_dict = {"Año": chile.index.tolist(), "Inmigrantes": chile.values.tolist()}
datos_cl = pd.DataFrame(cl_dict)

In [3]:
fig = px.line(datos_cl, x='Año', y='Inmigrantes')

fig.update_layout(width=1000, height=500, xaxis={'tickangle':-45})

fig.show()

# Añadiendo rótulos y modificando fuentes

In [4]:
fig = px.line(datos_cl, 
              x="Año", 
              y="Inmigrantes", 
              title='Inmigración de chilenos hacia Canadá (1980 - 2013)')

fig.update_layout(width=1000, height=500, 
                  xaxis={"tickangle": -45}, 
                  xaxis_title='Año', 
                  yaxis_title='Número de Inmigrantes',
                  font_family='Arial',
                  font_size=12,
                  font_color='gray'
                  )

fig.show()

# Alterando colores con Plotly

In [5]:
fig = px.line(
    datos_cl,
    x="Año",
    y="Inmigrantes",
    title="Inmigración de chilenos hacia Canadá (1980 - 2013)",
)

# Modificando la línea
fig.update_traces(line_color='green', line_width=3)

fig.update_layout(
    width=1000,
    height=500,
    xaxis={"tickangle": -45},
    xaxis_title="Año",
    yaxis_title="Número de Inmigrantes",
    font_family="Arial",
    font_size=12,
    font_color="gray",
)

fig.show()

In [6]:
sudamerica = df.query('Region == "América del Sur"')

In [7]:
# limpiar dataframe
sudamerica.drop(['Continente', 'Region', 'Total'], axis=1, inplace=True)

/var/folders/45/nnwdvrcs63j3c_75jxv__q_80000gn/T/ipykernel_21342/1641125021.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
sudamerica = sudamerica.T

In [9]:
sudamerica.head()

Pais,Argentina,Bolivia,Brasil,Chile,Colombia,Ecuador,Guyana,Paraguay,Perú,Surinam,Uruguay,Venezuela
1980,368,44,211,1233,266,238,2334,45,317,15,128,103
1981,426,52,220,1069,326,207,2943,26,456,10,132,117
1982,626,42,192,1078,360,184,3575,32,401,21,146,174
1983,241,49,139,781,244,155,2650,40,241,12,105,124
1984,237,38,145,681,235,179,1932,48,306,5,90,142


In [10]:
fig = px.line(
    sudamerica,
    x=sudamerica.index,
    y=sudamerica.columns,
    title="Inmigración de latinoamericanos hacia Canadá (1980 - 2013)",
    color='Pais',
    markers=True
)

fig.update_layout(
    width=1000,
    height=500,
    xaxis={"tickangle": -45},
    xaxis_title="Año",
    yaxis_title="Número de Inmigrantes",
    font_family="Arial",
    font_size=12,
    font_color="gray",
)

fig.show()

# Exportando en HTML

In [11]:
fig = px.line(
    sudamerica,
    x=sudamerica.index,
    y=sudamerica.columns,
    title="Inmigración de latinoamericanos hacia Canadá (1980 - 2013)",
    color="Pais",
    markers=True,
)

fig.update_layout(
    width=1000,
    height=500,
    xaxis={"tickangle": -45},
    xaxis_title="Año",
    yaxis_title="Número de Inmigrantes",
    font_family="Arial",
    font_size=12,
    font_color="gray",
)

fig.write_html('exported_data/inmigracion_latam_can.html')

fig.show()

# Animación de gráficos

In [12]:
# cambiar tipo de dato a Int
datos_cl['Año'] = datos_cl['Año'].astype(int)

In [14]:
import plotly.graph_objs as go

# crear figura vacía con función go.Figure()
fig = go.Figure()

# agregar línea (o trazo) al gráfico
# para que grafico se muestre sin linea antes de hacer click en play, se usa iloc[0]
# iloc[0] selecciona el primer valor de las columnas año e inmigrantes. Inicialmente solo mostrará ese punto.
fig.add_trace(
    go.Scatter(
        x=[datos_cl["Año"].iloc[0]],
        y=[datos_cl["Inmigrantes"].iloc[0]],
        mode="lines",  # modo de visualización
        name="Inmigrantes",
        line=dict(width=4),  # grosor de líneas
    )
)

fig.update_layout(
    title=dict(
        text="<b>Inmigración de Chilenos hacia Canadá en el periodo de 1980 a 2013</b>",
        x=0.12,  # posición horizontal del título en relación al ancho de la figura
        xanchor="left",  # alineación horizontal del título
        font=dict(size=20),  # tamaño de fuente del título
    ),
    xaxis=dict(
        range=[1980, 2013],  # rango del eje
        autorange=False,  # define si los límites se autoajustan automáticamente
        title="<b>Año</b>",
    ),
    yaxis=dict(range=[0, 7000], autorange=False, title="<b>Número de Inmigrantes</b>"),
    # botón play
    updatemenus=[
        dict(
            type="buttons", # menú estará compuesto por botones
            showactive=False, # ningún botón estará activo inicialmente
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[
                        None, # ninguna capa del elemento será afectada por la animación
                        {
                            "frame": {"duration": 100, "redraw": True},
                            "fromcurrent": True,
                        },
                    ],
                )
            ],
        )
    ],
    width=1000,
    height=500
)

# configuración de la animación
frames = [go.Frame(data=[go.Scatter(x=datos_cl['Año'].iloc[:i+1], y=datos_cl['Inmigrantes'].iloc[:i+1])]) for i in range(len(datos_cl))]

fig.frames = frames

fig.show()